In [1]:
from utils_v10 import *

In [2]:
input_df = pd.read_csv('/home/tobamo/analize/model-tobamo/notebooks/ver10/results/training/test_input_df.csv')
input_df

,orf_name,outgroup_cp_identity_score_min,outgroup_cp_identity_score_max,outgroup_cp_identity_score_mean,outgroup_cp_gap_openings_min,outgroup_cp_gap_openings_max,outgroup_cp_gap_openings_mean,outgroup_cp_gap_ratio_min,outgroup_cp_gap_ratio_max,outgroup_cp_gap_ratio_mean,...,tobamo_orf2_N_max,tobamo_orf2_N_mean,tobamo_orf2_aln_len_min,tobamo_orf2_aln_len_max,tobamo_orf2_aln_len_mean,contig_name,orf_len,contig_length,orf_acc_method,stop_codons
0,AB003936.1_start-0_len-2378_NT_ORF.1,0.186441,0.357143,0.271876,9,20,13.666667,0.035433,0.068729,0.049795,...,265,248.471698,324,500,462.452830,AB003936.1_start-0_len-2378_NT,274,2378,0,0
1,AB003936.1_start-0_len-2378_NT_ORF.2,0.228916,0.340136,0.269877,9,17,14.047619,0.030100,0.058824,0.046543,...,312,311.509434,302,459,314.415094,AB003936.1_start-0_len-2378_NT,312,2378,0,0
2,AB003936.1_start-0_len-2378_NT_ORF.3,0.209150,0.331034,0.260556,7,15,11.095238,0.033175,0.071090,0.053264,...,179,170.735849,285,502,416.094340,AB003936.1_start-0_len-2378_NT,179,2378,0,0
3,AB003936.1_start-0_len-2378_NT_ORF.4,0.218750,0.400000,0.284218,5,10,7.857143,0.028986,0.054217,0.044975,...,104,102.641509,189,461,404.849057,AB003936.1_start-0_len-2378_NT,104,2378,0,0
4,AB003936.1_start-0_len-2378_NT_ORF.5,0.198276,0.333333,0.266370,6,12,8.857143,0.037838,0.071429,0.051182,...,123,119.584906,287,486,395.603774,AB003936.1_start-0_len-2378_NT,123,2378,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24191,Z97873.1_start-965_len-661_aa_frame2,0.167742,0.300000,0.253464,6,17,11.476190,0.025862,0.072650,0.048144,...,219,204.566038,306,503,448.245283,Z97873.1_start-965_len-661,219,661,1,0
24192,Z97873.1_start-965_len-661_aa_frame3,0.198529,0.328244,0.254054,8,16,11.333333,0.034632,0.061776,0.048683,...,203,185.245283,306,519,445.132075,Z97873.1_start-965_len-661,219,661,1,16
24193,Z97873.1_start-965_len-661_aa_frame4,0.198630,0.325581,0.262462,6,16,11.714286,0.026549,0.062500,0.048104,...,208,190.452830,316,501,441.905660,Z97873.1_start-965_len-661,220,661,1,8
24194,Z97873.1_start-965_len-661_aa_frame5,0.196532,0.319728,0.261728,5,17,11.190476,0.023041,0.069106,0.048227,...,208,191.169811,323,500,453.113208,Z97873.1_start-965_len-661,219,661,1,11


In [ ]:
inv_nt_species2id_path = "/home/tobamo/analize/model-tobamo/notebooks/playground/results/inv_nt_dict.json"
inv_type_acc_dict_path = "/home/tobamo/analize/model-tobamo/notebooks/playground/results/inv_type_acc_dict.json"
orf_fasta_path = "/home/tobamo/analize/model-tobamo/notebooks/ver7/results/combined_orfs.fasta"


def add_info_to_training_input_df(
    input_df, batch_no: int, inv_nt_species2id_path: str, inv_type_acc_dict_path: str, orf_fasta_path: str
):

    if "contig_name" not in input_df.columns:
        input_df["contig_name"] = input_df["orf_name"].str.extract(r"(.*)(?:_ORF\.\d+|_aa_frame\d+)")
    if batch_no == 0:
        input_df["nt_id"] = input_df["contig_name"]
    else:
        input_df["nt_id"] = input_df["orf_name"].str.extract(r"(.*)_start")

    # add species info
    inv_nt_species2id = mpu.io.read(inv_nt_species2id_path)
    input_df["species"] = input_df["nt_id"].map({k: v[0] for k, v in inv_nt_species2id.items()})

    # add orf_type info
    inv_type_acc_dict = mpu.io.read(inv_type_acc_dict_path)
    input_df["orf_type"] = input_df["nt_id"].map({k: v[0] for k, v in inv_type_acc_dict.items()})

    # add strand orientation info
    with open(orf_fasta_path, "r") as file:
        orf_dict = {
            record.description.split()[0]: "REVERSE" if "(-)" in record.description else "FORWARD"
            for record in SeqIO.parse(file, "fasta")
        }
    input_df["strand"] = input_df["orf_name"].map(orf_dict)

    return input_df